# Analyze dwell times (notebook 2)

This notebook performs positional KS tests on dwell times from the Trizol OLD and f1f2_GL Tombo runs.

The file will produce a dataframe with a row for every position. The dataframe will have these columns:
* IVT median dwell-time
* Trizol dwell-time
* IVT coverage depth
* Trizol coverage depth
* KS test $p$-value
* KS test statistic
* U test $p$-value
* U test statistic

Rows will be labeled by zero-indexed nucleotide position on the reference genome.

**Input:** two CSV files containing per-read per-nucleotide dwell times extracted from fast5 files

**Output:** CSV file describing and comparing dwell-time distributions on a per-position basis

In [1]:
from os import environ, path
import shutil

import numpy as np
import pandas as pd
from scipy import stats

To make things faster, let's try copying our input data onto our compute node's local filesystem.

In [2]:
def copy_to_node(src_path):
    '''
    Copy a file to local storage on the current compute node. (I don't know
    if this would work for Jupyter Notebook sessions running on multiple
    nodes.)
    
    Args:
        src_path (str): filepath to the file to be copied
    
    Returns:
        str: Absolute path to the new file location
    '''
    src_basename = path.basename(src_path)
    node_path = environ['TMPDIR'] # environment variable (local node directory)
    dest_path = path.join(node_path, src_basename)
    shutil.copyfile(src_path, dest_path)
    return dest_path

In [3]:
ivt_path = "/fs/project/PAS1405/extract_dwells_3/f1f2_GL_dwells.csv"
trizol_path = "/fs/project/PAS1405/extract_dwells_3/Trizol_OLD_8K_single_fast5_dwells.csv"

ivt_path = copy_to_node(ivt_path)
trizol_path = copy_to_node(trizol_path)

print(ivt_path)
print(trizol_path)

/tmp/pbstmp.10935695/f1f2_GL_dwells.csv
/tmp/pbstmp.10935695/Trizol_OLD_8K_single_fast5_dwells.csv


In [4]:
def load_dwell_csv(csv_path):
    '''
    Load a CSV of per-read per-position dwell times into a dataframe.
    
    Args:
        csv_path (str): path to the CSV file
    
    Returns:
        pd.DataFrame: a dataframe containing FLOAT dwell times
            * rows labeled by read_id (str)
            * columns labeled by genomic position number (int)
    '''
    retval = (
        pd.read_csv(csv_path, header=0, index_col=0)
        .rename_axis('read_id', axis=0)
        .rename_axis('pos', axis=1)
    )
    retval.columns = retval.columns.astype(int)
    return retval

In [5]:
# This cell takes about 5 minutes on one full Owens cluster node.

trizol_dwells = load_dwell_csv(trizol_path)
print('trizol_dwells loaded')
ivt_dwells = load_dwell_csv(ivt_path)
print('ivt_dwells loaded')
print('done')

trizol_dwells loaded
ivt_dwells loaded
done


In [6]:
pos_intersection = ivt_dwells.columns.intersection(trizol_dwells.columns)
pos_intersection

Int64Index([   7,    8,    9,   10,   11,   12,   13,   14,   15,   16,
            ...
            9161, 9162, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170],
           dtype='int64', name=u'pos', length=9164)

In [7]:
trizol_depth = trizol_dwells.count().rename('trizol_depth')
ivt_depth =    ivt_dwells.count().rename('ivt_depth')

trizol_median = trizol_dwells.median(axis=0, skipna=True).rename('trizol_median')
ivt_median =    ivt_dwells.median(axis=0, skipna=True).rename('ivt_median')

In [12]:
ks_stat = pd.Series(index=pos_intersection, name='ks_stat')
ks_pval = pd.Series(index=pos_intersection, name='ks_pval')
u_stat = pd.Series(index=pos_intersection, name='u_stat')
u_pval = pd.Series(index=pos_intersection, name='u_pval')

for pos in pos_intersection:
    trizol_col = trizol_dwells.loc[:, pos].dropna()
    ivt_col = ivt_dwells.loc[:, pos].dropna()
    
    ks_stat.loc[pos], ks_pval.loc[pos] = stats.ks_2samp(trizol_col, ivt_col)
    u_stat.loc[pos], u_pval.loc[pos] = stats.mannwhitneyu(trizol_col, ivt_col, alternative='two-sided')

In [32]:
u_effect_size = (u_stat/(ivt_depth*trizol_depth)).rename('u_effect_size')

ks_effect_size = (
    ks_stat
    * (ivt_depth + trizol_depth)**(0.5)
    / (ivt_depth * trizol_depth)**(0.5)
).rename('ks_effect_size')

In [33]:
distributional_dwell_stats = pd.concat(
    [trizol_depth,
     ivt_depth,
     trizol_median,
     ivt_median,
     ks_stat,
     ks_pval,
     ks_effect_size,
     u_stat,
     u_pval,
     u_effect_size],
    axis=1
)
distributional_dwell_stats

,trizol_depth,ivt_depth,trizol_median,ivt_median,ks_stat,ks_pval,ks_effect_size,u_stat,u_pval,u_effect_size
pos,,,,,,,,,,
1,NaN,8514,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,8888,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,9032,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,10111,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,21374,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,23840,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,32019,2.0,58.0,0.881789,1.900674e-01,0.881803,1892.5,1.262934e-01,0.059106
8,3.0,39654,13.0,29.0,0.592500,1.520671e-01,0.342093,32916.5,1.798000e-01,0.276698
9,7.0,41306,12.0,18.0,0.232688,7.870692e-01,0.087955,124597.5,5.257258e-01,0.430921


In [34]:
distributional_dwell_stats.to_csv(
    'distributional_dwell_stats.csv',
    index_label=True
)